In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import emoji
from gensim.models import Word2Vec
import torch
import time

In [100]:
UNK = "<unk>"
empty = "<empty>"
wordEmbSize = 64
data = pd.read_csv("data3.csv")

split = np.random.rand(len(data)) < 0.8
trainData = data[split]
testData = data[~split]

# Data Preprocessing

+ Cleaning by using nltk word tokenizer and lemmatizer
+ Adds spaces to emojis to separate them to different words using emoji library's re
+ Add a start and end token
+ Build vocab for words
+ Build vocab for emojis
+ Makes labels as 0 or 1 for each word. If label is 1, means that word is followed by an emoji

In [144]:
RE_EMOJI = emoji.get_emoji_regexp()
tokenizer = nltk.word_tokenize
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# tokens normalized
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#converting text to words
def preprocessing(data, train=True):
    newData = {"words":[], "labels":[]}
    for text in data["texts"]:
        #converting to words
        emoji_split = RE_EMOJI.split(text)
        emoji_split = [x.strip() for x in emoji_split if x]
        text = " ".join(emoji_split)
        textWords = LemNormalize(text)
        textWords.insert(0,"<s>")
        textWords.append("</s>")
        newData["words"].append(textWords)
        
        #getting labels
        labels = []
        if train:
            for i in range(1, len(textWords)):
                word = textWords[i]
                if RE_EMOJI.match(word):
                    labels.append(1)
                else:
                    labels.append(0)
            labels.append(0)
        else:
            labels = [0 * len(textWords)]
        newData["labels"].append(labels)
    return pd.DataFrame(newData)

def make_vocabs(data):
    vocab = set()
    vocab.add(UNK)
    emojiVocab = set()
#     emojiVocab.add(empty)
    for text in data["words"]:
        for word in text:
            vocab.add(word)
            if RE_EMOJI.match(word):
                emojiVocab.add(word)
    return vocab, emojiVocab

def remove_unk(data, vocab):
    updatedData = []
    for text in data["words"]:
        updatedWords = [word if word in vocab else UNK for word in text]
        updatedData.append(updatedWords)
    data["words"] = updatedData
    return data

train = preprocessing(trainData, True)
vocab, emojiVocab = make_vocabs(train)
vocabIdx = {word : i for i, word in enumerate(vocab)}
eVocabIdx = {emoji : i for i, emoji in enumerate(emojiVocab)}

test = preprocessing(testData, True)
test = remove_unk(test, vocab)

# Word Embeddings

+ Using gensim's Word2Vec
+ Builds model with word embedding size specified earlier

In [5]:
def getEmbModel(data, vocab):
    docs = [[UNK]]
    docs.extend(data["words"])
    model = Word2Vec(docs, min_count = 1, size = wordEmbSize)
    print(model)
    return model

# Takes: dataset, word2vec model, and vocabulary from training
# returns: list of tuples. First value is a torch of word embeddings for that sentence,
# second value is the labels for each word
def getEmb(data, model, vocab):
    vecData = []
    for text,y in zip(data["words"],data["labels"]):
        wordEmb = []
        for word in text:
            if word in vocab:
                wordEmb.append(model[word])
            else:
                wordEmb.append(model[UNK])
        wordEmb = torch.FloatTensor(wordEmb)
        vecData.append((wordEmb, y))
    return vecData

model = getEmbModel(train, vocab)
trainEmb = getEmb(train, model, vocab)

Word2Vec(vocab=19720, size=64, alpha=0.025)


/Users/ericsun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [6]:
sample = train.sample(5)
for index,row in sample.iterrows():
    print(row["words"])
    print(row["labels"])
    print(trainEmb[index][0])

['<s>', 'wyd', 'want', 'ya', 'as', 'whoop', 'peep', 'the', 'champion', 'ship', 'kid', 'i', 'll', 'beat', 'your', 'as', 'no', 'regrits', 'i', 'll', 'slap', 'your', 'dimb', 'as', 'back', 'to', 'the', 'foreign', 'land', 'u', 'came', 'from', 'bitch', 'i', '’', 'm', 'the', 'champion', 'and', 'you', '’', 're', 'just', 'a', 'faggot', 'silly', 'as', 'bitch', 'you', 'don', '’', 't', 'won', '’', 't', 'none', 'u', 'ain', '’', 't', 'want', 'none', 'ah', 'shit', 'ah', 'shit', 'u', 'gone', 'get', 'the', 'smoke', 'u', 'gone', 'get', 'the', 'smoke', 'better', 'run', 'big', 'homie', '🙏', 'ima', 'be', 'wildin', '💪🏼', '💪🏼', '</s>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]
tensor([[ 0.7273,  0.1699,  0.9712,  ..., -0.2124, -0.4577, -0.6436],
        [ 0.0443, -0.0031,  0.0582,  ..., -0.01

# Building supervised models to predict next emoji

+ RNN based architecture where we look at the hidden layer for every word
  + Using hidden layer, predict if is an emoji and what emoji it is
+ Asked TA from NLP class, they said this is similar to a language modelling problem where we only predict the set of emoji vocabulary
  + Could also view as sequence labelling where tag is next emoji or no emoji

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

jupyter = True
if jupyter:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

# Feedforward NN

+ https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
+ Built using previous couple of words
+ Feature is word embeddings using pytorch nn.Embedding. Use vocabulary and map words to index and emojis to index. Entire vocabulary is fed as input to the NN but output of NN is either emoji or no emoji

In [116]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, context_size, hidden):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden)
        self.activation = nn.ReLU()
        self.softmax = nn.LogSoftmax()
        self.loss = nn.NLLLoss()
        self.linear2 = nn.Linear(hidden, output_size)

    def compute_loss(self, predicted_vector, label):
        return self.loss(predicted_vector, label)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = self.activation(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = self.softmax(out)
        return log_probs

In [120]:
def FFfeatures(data, numContext = 2):
    grams = []
    numEmpty = 0
    total = 0
    for text in data["words"]:
        curr = []
        for i in range(len(text) - numContext):
            predictWord = text[i+numContext]
            if not RE_EMOJI.match(predictWord):
                predictWord = 0
                numEmpty += 1
            else:
                predictWord = 1
            total += 1
            context = [word for word in text[i:i+numContext]]
            curr.append((context, predictWord))
        grams.append(curr)
    print("% empty:{}".format(numEmpty / total))
    return grams

In [124]:
CONTEXT_SIZE = 4
train_feats = FFfeatures(train, numContext=CONTEXT_SIZE)
test_feats = FFfeatures(test, numContext=CONTEXT_SIZE)

% empty:0.7183500017066594
% empty:0.7298338285462749


In [125]:
# FFNN with context to predict if emoji or not
EMBEDDING_DIM = 50

model = NGramLanguageModeler(len(vocab), 2, EMBEDDING_DIM, CONTEXT_SIZE, 128)
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    print("Training started for epoch:{}".format(epoch + 1))
    random.shuffle(train_feats)
    start_time = time.time()
    correct = 0
    total = 0
    guess0 = 0
    guess1 = 0
    minibatch_size = 16
    N = len(train_feats)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for idx in range(minibatch_size):
            text = train_feats[minibatch_idx * minibatch_size + idx]
            for context, target in text:
                context_idx = torch.tensor([vocabIdx[w] for w in context], dtype=torch.long)
                log_probs = model(context_idx)
                idx_loss = model.compute_loss(log_probs, torch.tensor([target]))
                
                if loss is None:
                    loss = idx_loss
                else:
                    loss += idx_loss
                predicted_label = torch.argmax(log_probs)
                if predicted_label == 0:
                    guess0 += 1
                else:
                    guess1 += 1
                correct += int(predicted_label == target)
                total += 1
        loss = loss / minibatch_size
        loss.backward()
        optimizer.step()
    print("Training completed for epoch:{}".format(epoch + 1))
    print("Time for train:{}".format(time.time() - start_time))
    print("Accuracy:{}".format(correct / total))
    print("Guessed not emoji:{} Guessed emoji:{}".format(guess0, guess1))
    
    #validation
    model.eval()
    optimizer.zero_grad()
    print("Validation started for epoch:{}".format(epoch + 1))
    random.shuffle(test_feats)
    start_time = time.time()
    correct = guess0 = guess1 = total = 0
    minibatch_size = 16
    N = len(test_feats)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        for idx in range(minibatch_size):
            text = test_feats[minibatch_idx * minibatch_size + idx]
            for context, target in text:
                context_idx = torch.tensor([vocabIdx[w] for w in context], dtype=torch.long)
                log_probs = model(context_idx)

                predicted_label = torch.argmax(log_probs)
                if predicted_label == 0:
                    guess0 += 1
                else:
                    guess1 += 1
                correct += int(predicted_label == target)
                total += 1
    print("Validation completed for epoch:{}".format(epoch + 1))
    print("Time for validation:{}".format(time.time() - start_time))
    print("Accuracy:{}".format(correct / total))
    print("Guessed not emoji:{} Guessed emoji:{} %Emoji:{}".format(guess0, guess1, guess1/(guess1+guess0)))

Training started for epoch:1


/Users/ericsun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Training completed for epoch:1
Time for train:392.81811571121216
Accuracy:0.6136874267198807
Guessed not emoji:130887 Guessed emoji:42250
Validation started for epoch:1



Validation completed for epoch:1
Time for validation:9.102935075759888
Accuracy:0.27813709912047085
Guessed not emoji:903 Guessed emoji:43780 %Emoji:0.9797909719580153
Training started for epoch:2



Training completed for epoch:2
Time for train:397.34837198257446
Accuracy:0.681239981086598
Guessed not emoji:156974 Guessed emoji:16448
Validation started for epoch:2


...


Training completed for epoch:3
Time for train:672.4186429977417
Accuracy:0.7171286466026587
Guessed not emoji:173312 Guessed emoji:0
Validation started for epoch:3



Validation completed for epoch:3
Time for validation:9.07215166091919
Accuracy:0.7309490259231922
Guessed not emoji:44709 Guessed emoji:0 %Emoji:0.0
Training started for epoch:4



Training completed for epoch:4
Time for train:395.56516695022583
Accuracy:0.7185738719193728
Guessed not emoji:174234 Guessed emoji:0
Validation started for epoch:4



Validation completed for epoch:4
Time for validation:9.60697317123413
Accuracy:0.729536887452881
Guessed not emoji:44568 Guessed emoji:0 %Emoji:0.0
Training started for epoch:5


KeyboardInterrupt: 

# RNN/LSTM 

+ https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py
+ 2 models
  + one to determine whether a word is an emoji (treated as a seq labelling task)
  + one to determine which emoji it is (also treated as seq labelling)

## Model for predicting if Emoji or not

In [146]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, loss_weights, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.softmax = nn.LogSoftmax()
        self.loss = nn.NLLLoss(weight=loss_weights)
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
    def compute_loss(self, predicted_vector, label):
        return self.loss(predicted_vector, label)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = self.softmax(tag_space)
        return tag_scores

In [147]:
def determineEmojiFeats(data):
    feats = []
    for text,label in zip(data["words"], data["labels"]):
        feats.append((text, label))
    return feats

train_feats = determineEmojiFeats(train)
test_feats = determineEmojiFeats(test)

In [148]:
EMBEDDING_DIM = 50
HIDDEN_DIM = 128

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, torch.tensor([0.5,1]), len(vocab), 2)
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    print("Training started for epoch:{}".format(epoch + 1))
    random.shuffle(train_feats)
    start_time = time.time()
    correct = guess0 = guess1 = total = 0
    minibatch_size = 16
    N = len(train_feats)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for idx in range(minibatch_size):
            text, label = train_feats[minibatch_idx * minibatch_size + idx]
            label = torch.tensor(label, dtype=torch.long)
            text_idx = torch.tensor([vocabIdx[w] for w in text], dtype = torch.long)
            log_probs = model(text_idx)
            idx_loss = model.compute_loss(log_probs, label)
            if loss is None:
                loss = idx_loss
            else:
                loss += idx_loss
            for i in range(len(label)):
                corr_label = label[i]
                pred_label = torch.argmax(log_probs[i])
                if pred_label == 0:
                    guess0 += 1
                else:
                    guess1 += 1
                correct += int(pred_label == corr_label)
                total += 1
        loss = loss / minibatch_size
        loss.backward()
        optimizer.step()
    print("Training completed for epoch:{}".format(epoch + 1))
    print("Time for train:{}".format(time.time() - start_time))
    print("Accuracy:{}".format(correct / total))
    print("Guessed not emoji:{} Guessed emoji:{} %Emoji:{}".format(guess0, guess1, guess1/(guess1+guess0)))
    
    #validation
    model.eval()
    optimizer.zero_grad()
    print("Validation started for epoch:{}".format(epoch + 1))
    random.shuffle(test_feats)
    start_time = time.time()
    correct = guess0 = guess1 = total = 0
    minibatch_size = 16
    N = len(test_feats)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        for idx in range(minibatch_size):
            text, label = test_feats[minibatch_idx * minibatch_size + idx]
            label = torch.tensor(label, dtype=torch.long)
            text_idx = torch.tensor([vocabIdx[w] for w in text], dtype = torch.long)
            log_probs = model(text_idx)
            for i in range(len(label)):
                corr_label = label[i]
                pred_label = torch.argmax(log_probs[i])
                if pred_label == 0:
                    guess0 += 1
                else:
                    guess1 += 1
                correct += int(pred_label == corr_label)
                total += 1
    print("Validation completed for epoch:{}".format(epoch + 1))
    print("Time for validation:{}".format(time.time() - start_time))
    print("Accuracy:{}".format(correct / total))
    print("Guessed not emoji:{} Guessed emoji:{} %Emoji:{}".format(guess0, guess1, guess1/(guess1+guess0)))

Training started for epoch:1


/Users/ericsun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


KeyboardInterrupt: 

## Model for predicting emoji

+ Still WIP
+ Problems: possible outcomes is too high and our data is too sparse to guess emojis correctly
+ Probably either need more data or narrow down data vocabulary significantly
+ Simple hacky solution: build a mapping of certain words to certain emojis

In [143]:
class LSTMPredictor(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.softmax = nn.LogSoftmax()
        self.loss = nn.NLLLoss()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to output
        self.hidden2tag = nn.Linear(hidden_dim, output_size)
        
    def compute_loss(self, predicted_vector, label):
        return self.loss(predicted_vector, label)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        _, hidden = self.lstm(embeds.view(len(sentence), 1, -1))
        hidden = hidden.contiguous().view(-1, self.hidden_dim)
        output = self.softmax(self.hidden2tag(hidden))
        return output

In [145]:
def emojiTypeFeats(data):
    feats = []
    for text in data["words"]:
        curr = []
        for wordIdx in range(len(text)):
            word = text[wordIdx]
            if RE_EMOJI.match(word):
                curr.append((text[:wordIdx], eVocabIdx[word]))
        feats.append(curr)
    return feats

train_feats = emojiTypeFeats(train)
test_feats = emojiTypeFeats(test)

In [137]:
EMBEDDING_DIM = 50
HIDDEN_DIM = 128

model = LSTMPredictor(EMBEDDING_DIM, HIDDEN_DIM, len(vocab), len(emojiVocab))
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    print("Training started for epoch:{}".format(epoch + 1))
    random.shuffle(train_feats)
    start_time = time.time()
    correct = guess0 = guess1 = total = 0
    minibatch_size = 16
    N = len(train_feats)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for idx in range(minibatch_size):
            text = train_feats[minibatch_idx * minibatch_size + idx]
            for context, label in text:
                label = torch.tensor(label, dtype=torch.long)
                text_idx = torch.tensor([vocabIdx[w] for w in text], dtype = torch.long)
                log_probs = model(text_idx)
                idx_loss = model.compute_loss(log_probs, label)
                if loss is None:
                    loss = idx_loss
                else:
                    loss += idx_loss
                for i in range(len(label)):
                    corr_label = label[i]
                    pred_label = torch.argmax(log_probs[i])
                    if pred_label == 0:
                        guess0 += 1
                    else:
                        guess1 += 1
                    correct += int(pred_label == corr_label)
                    total += 1
        loss = loss / minibatch_size
        loss.backward()
        optimizer.step()
    print("Training completed for epoch:{}".format(epoch + 1))
    print("Time for train:{}".format(time.time() - start_time))
    print("Accuracy:{}".format(correct / total))
    print("Guessed not emoji:{} Guessed emoji:{} %Emoji:{}".format(guess0, guess1, guess1/(guess1+guess0)))
    
    #validation
    model.eval()
    optimizer.zero_grad()
    print("Validation started for epoch:{}".format(epoch + 1))
    random.shuffle(test_feats)
    start_time = time.time()
    correct = guess0 = guess1 = total = 0
    minibatch_size = 16
    N = len(test_feats)
    for minibatch_idx in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        for idx in range(minibatch_size):
            text = test_feats[minibatch_idx * minibatch_size + idx]
            for context, label in text:
                label = torch.tensor([label], dtype=torch.long)
                text_idx = torch.tensor([vocabIdx[w] for w in text], dtype = torch.long)
                log_probs = model(text_idx)
                for i in range(len(label)):
                    corr_label = label[i]
                    pred_label = torch.argmax(log_probs[i])
                    if pred_label == 0:
                        guess0 += 1
                    else:
                        guess1 += 1
                    correct += int(pred_label == corr_label)
                    total += 1
    print("Validation completed for epoch:{}".format(epoch + 1))
    print("Time for validation:{}".format(time.time() - start_time))
    print("Accuracy:{}".format(correct / total))
    print("Guessed not emoji:{} Guessed emoji:{} %Emoji:{}".format(guess0, guess1, guess1/(guess1+guess0)))

Training started for epoch:1


/Users/ericsun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Training completed for epoch:1
Time for train:147.91556906700134
Accuracy:0.052651767508529665
Validation started for epoch:1



Validation completed for epoch:1
Time for validation:3.2849442958831787
Accuracy:0.0037294878170064643
Guessed not emoji:0 Guessed emoji:12066 %Emoji:1.0
Training started for epoch:2



Training completed for epoch:2
Time for train:152.59969925880432
Accuracy:0.0007692930601668151
Validation started for epoch:2


...


Training completed for epoch:3
Time for train:145.50066494941711
Accuracy:0.0003434620979473089
Validation started for epoch:3



Validation completed for epoch:3
Time for validation:3.6287479400634766
Accuracy:0.00352834988102076
Guessed not emoji:12187 Guessed emoji:0 %Emoji:0.0
Training started for epoch:4



Training completed for epoch:4
Time for train:144.32569098472595
Accuracy:0.0003451706564333719
Validation started for epoch:4



Validation completed for epoch:4
Time for validation:3.388035774230957
Accuracy:0.0035824377239023577
Guessed not emoji:12003 Guessed emoji:0 %Emoji:0.0
Training started for epoch:5


KeyboardInterrupt: 